In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import warnings

from matplotlib import rc

import mplcyberpunk
plt.style.use('cyberpunk')
warnings.filterwarnings("ignore")

rc('font', family='AppleGothic') 			
plt.rcParams['axes.unicode_minus'] = False
mpl.rc('font', size=12)
%matplotlib inline

In [30]:
data_path = '/Users/tastebread/Desktop/kaggle/bike-sharing-demand/'
train = pd.read_csv(data_path + 'train.csv')
test = pd.read_csv(data_path + 'test.csv')
samplesubmission = pd.read_csv(data_path + 'sampleSubmission.csv')

In [31]:
#앞서 봤던 weather 피쳐에서 4가 아닌 데이터만 추출하기
train = train[train['weather'] != 4]

In [32]:
#train,test에 공통으로 반영해야하므로 train,test 합쳤다가 다시 분리해주기

df = pd.concat([train,test],ignore_index=True)
df.head(-5)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3.0,13.0,16.0
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8.0,32.0,40.0
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5.0,27.0,32.0
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3.0,10.0,13.0
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
17368,2012-12-31 14:00:00,1,0,1,2,11.48,13.635,45,15.0013,NaN,NaN,NaN
17369,2012-12-31 15:00:00,1,0,1,2,11.48,14.395,45,8.9981,NaN,NaN,NaN
17370,2012-12-31 16:00:00,1,0,1,2,10.66,12.880,48,12.9980,NaN,NaN,NaN
17371,2012-12-31 17:00:00,1,0,1,2,10.66,14.395,48,6.0032,NaN,NaN,NaN


In [33]:
from datetime import datetime
import calendar
"""
판다스 apply 함수를 사용해서 datetime 로직에 적용해 날짜,연도,월,일,시,분,초 피처 생성
"""
#df['date'] = df['datetime'].apply(lambda x: x.split()[0])

#연도,월,일,시,분,초
"""
df['year'] = df['datetime'].apply(lambda x: x.split()[0].split("-")[0])
df['month'] = df['datetime'].apply(lambda x: x.split()[0].split("-")[1])
df['day'] = df['datetime'].apply(lambda x: x.split()[0].split("-")[2])
df['hour'] = df['datetime'].apply(lambda x: x.split()[1].split(":")[0])
df['Weekday'] = df['date'].apply(lambda dateString: calendar.day_name[datetime.strptime(dateString,"%Y-%m-%d").weekday()])
"""

df['datetime'] = pd.to_datetime(df['datetime'])
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['hour'] = df['datetime'].dt.hour
df['weekday'] = df['datetime'].dt.weekday
df.head(5)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,hour,weekday
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3.0,13.0,16.0,2011,1,0,5
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8.0,32.0,40.0,2011,1,1,5
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5.0,27.0,32.0,2011,1,2,5
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3.0,10.0,13.0,2011,1,3,5
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0.0,1.0,1.0,2011,1,4,5


In [35]:
#이제 필요 없는 피쳐 제거 test데이터에 없는 casual,registered 삭제
#date 피쳐가 갖는 정보는 다른 피처들에게도 담겨 있으므로 제거, datetime 도 제거
#windspeed는 히트맵으로 분석했을때 상관계수가 0.1로 낮으므로 제거
#month는 대분류의 성격이라 month 피처도 제거

drop = ['casual','registered','month','datetime','windspeed']

df = df.drop(drop,axis=1)


In [36]:
#합쳤던 train,test 데이터를 다시 나눠주기

#타깃값이 있으면 훈련 데이터, 타깃값이 없으면 테스트 데이터
# pd.isnull 앞에 부정의 의미 ~를 써서 Null이 아니면 훈련데이터 null이면 테스트 데이터로 만들어준다.
X_train = df[~pd.isnull(df['count'])] 
X_test = df[pd.isnull(df['count'])]

X_train = X_train.drop(['count'],axis=1)
X_test = X_test.drop(['count'],axis=1)

y = train['count']




In [37]:
X_train.head(5)

,season,holiday,workingday,weather,temp,atemp,humidity,year,hour,weekday
0,1,0,0,1,9.84,14.395,81,2011,0,5
1,1,0,0,1,9.02,13.635,80,2011,1,5
2,1,0,0,1,9.02,13.635,80,2011,2,5
3,1,0,0,1,9.84,14.395,75,2011,3,5
4,1,0,0,1,9.84,14.395,75,2011,4,5


In [38]:
#훈련이 제대로 이루어졌는데 확인하기위해 평가지표를 만들기 이번 경진대회 평가지표인 RMSLE를 계산하는 함수 제작

def rmsle(y_true,y_pred, convertExp=True): #y_true -> 실제 타겟값, y_pred -> 예측값, convertExp -> 입력 데이터를 지수변환할지를 정하는 파라미터 True면 지수변환함
    #지수 변환 %이유는 타겟값이 count 가 아닌 log(count)기 때문
    if convertExp:
        y_true = np.exp(y_true)
        y_pred = np.exp(y_pred)

    #로그 변환후 결측값을 0으로 변환
    log_true = np.nan_to_num(np.log(y_true+1)) #np.nan_to_num()함수는 NaN 결측값을 모두 0으로 바꾸는 기능.
    log_pred = np.nan_to_num(np.log1p(y_pred)) #log(y+1)은 log1p(y)로 변경 가능하다

    #RMSLE 계산
    output = np.sqrt(np.mean((log_true - log_pred)**2))
    return output

In [46]:
#모델 훈련
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
lrg_model = LinearRegression()

log_y = np.log(y) #타겟값 로그 변환

lrg_model.fit(X_train,log_y) #훈련


LinearRegression()

In [47]:
from lightgbm import LGBMRegressor

lgbm_params = {'n_estimators':[1000]}
lgbm_reg = LGBMRegressor(n_estimators=1000, learning_rate=0.05,num_leaves=4,subsample=0.6,colsample_bytree=0.4,reg_lambda=10,n_jobs=-1)

lgbm_reg.fit(X_train,log_y)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001261 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 239
[LightGBM] [Info] Number of data points in the train set: 10885, number of used features: 10
[LightGBM] [Info] Start training from score 4.552511


LGBMRegressor(colsample_bytree=0.4, learning_rate=0.05, n_estimators=1000,
              n_jobs=-1, num_leaves=4, reg_lambda=10, subsample=0.6)

In [48]:
lgbm_reg_pred = lgbm_reg.predict(X_train)
print(f'선형 회귀의 RMSLE 값 : {rmsle(log_y,lgbm_reg_pred,True):.4f}')

선형 회귀의 RMSLE 값 : 0.3320


In [43]:
#이건 잘못된 방법 구현만 알기위해서 써놓음
preds = lrg_model.predict(X_train)
print(f'선형 회귀의 RMSLE 값 : {rmsle(log_y,preds,True):.4f}')



선형 회귀의 RMSLE 값 : 1.0205


In [45]:
"""
kaggle 제출시 주의 사항

1.테스트 데이터로 예측한 결과를 이용해야한다.
2.예측한 값에 지수변환을 해줘야한다.
"""

lrg_pred = lrg_model.predict(X_test)

samplesubmission['count'] = np.exp(lrg_pred) # 지수변환을 필수로 해줘야함!
samplesubmission.to_csv('submission.csv',index=False)


In [49]:
lgbm_reg_pred = lgbm_reg.predict(X_test)

samplesubmission['count'] = np.exp(lgbm_reg_pred) # 지수변환을 필수로 해줘야함!
samplesubmission.to_csv('submission.csv',index=False)


In [50]:
#하이퍼파라미터를 최적화 하고 모델훈련을 하는 방법
#그리드서치로 릿지 회귀 모델 찾아내기

from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

ridge = Ridge()

#그리드 서치 객체 생성 릿지 모델에서 중요한 파라미터는 alpha로 값이 클수록 규제 강도가 세집니다.

ridge_params = {'max_iter' : [3000], 'alpha':[0.1,1,2,3,4,10,30,100,200,300,400,800,900,1000]}

#교차 검증용 평가 함수(RMSLE 점수 계산

rmsle_score = metrics.make_scorer(rmsle, greater_is_better=False)

#그리드서치 객체 생성
grid_ridge = GridSearchCV(estimator=ridge,
                          param_grid=ridge_params,
                          scoring=rmsle_score,
                          cv=5)


In [52]:
grid_ridge.fit(X_train,log_y)
print('최적 하이퍼 파라미터 : ',grid_ridge.best_params_)

최적 하이퍼 파라미터 :  {'alpha': 0.1, 'max_iter': 3000}


In [53]:
"""
이후 성능 검증은
그리드서치를 완료하면 그리드서치 객체의 best_estimator_ 속성에 최적 예측기가 저장되어있으므로 속성에 저장된 모델을 사용하면됨
"""

preds = grid_ridge.best_estimator_.predict(X_train)
print(f'선형 회귀의 RMSLE 값 : {rmsle(log_y,preds,True):.4f}')

선형 회귀의 RMSLE 값 : 1.0205


In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf_ml = RandomForestRegressor()